# Strongly Connected Components

Groups in a directed graph are said to be [strongly connected](https://en.wikipedia.org/wiki/Strongly_connected_component) if there exists a path between each node in the group.

In the graph below, there are three strongly connected components: `a b e`, `f g`, and `c d h`.



<img width="100%" height="100%" src="images/scc_graph.png">

In [1]:
graph = {'a': ['b'],
         'b': ['c', 'e', 'f'],
         'c': ['d', 'g'],
         'd': ['c', 'h'],
         'e': ['a', 'f'],
         'f': ['g'],
         'g': ['f'],
         'h': ['d', 'g']}

## Kosaraju's algorithm

[Wikipedia](https://en.wikipedia.org/wiki/Kosaraju%27s_algorithm) | [NetworkX Implementation](https://networkx.org/documentation/stable/_modules/networkx/algorithms/components/strongly_connected.html#kosaraju_strongly_connected_components)

In [6]:
from collections import defaultdict


def components(graph):
    transposed = defaultdict(list)
    
    for v in graph:
        for w in graph[v]:
            transposed[w].append(v)

    visited = set()
    nodes = []
    components = defaultdict(set)
    
    def visit(v):
        if v not in visited:
            visited.add(v)
            
            for w in graph[v]:
                visit(w)

            nodes.insert(0, v)
                
    for v in graph:
        visit(v)
    
    def assign(v, root):
        if v in visited:
            visited.remove(v)
            
            components[root].add(v)

            for w in transposed[v]:
                assign(w, root)      
                
       
    for v in nodes:
        assign(v, v)
        
    return list(components.values())


components(graph)

KeyError: 'b'

In [3]:
import networkx as nx


def nx_components(graph):
    graph = nx.DiGraph(graph)

    return list(nx.kosaraju_strongly_connected_components(graph))


mine = components(graph)
reference = nx_components(graph)

assert len(mine) == len(reference)

for component in mine:
    assert component in reference

## Tarjan's strongly connected components algorithm

[Wikipedia](https://en.wikipedia.org/wiki/Tarjan%27s_strongly_connected_components_algorithm) | [NetworkX Implementation](https://networkx.org/documentation/stable/_modules/networkx/algorithms/components/strongly_connected.html#strongly_connected_components)

In [4]:
from collections import defaultdict


def components(graph):
    index = 0            
    indexes = {}
    low_links = {}
    stack = []
    components = []

    def dfs(v):
        nonlocal index
        
        indexes[v] = low_links[v] = index
        index += 1

        stack.append(v)
        
        for w in graph[v]:
            if w not in indexes:
                dfs(w)
                low_links[v] = min(low_links[v], low_links[w])
            elif w in stack:
                low_links[v] = min(low_links[v], indexes[w])
                
        if low_links[v] == indexes[v]:
            component = set()
            w = None

            while w != v:
                w = stack.pop()
                component.add(w)

            components.append(component)
            
    for v in graph:
        if v not in indexes:
            dfs(v)
            
    return components


components(graph)

[{'f', 'g'}, {'c', 'd', 'h'}, {'a', 'b', 'e'}]

In [5]:
import networkx as nx


def nx_components(graph):
    graph = nx.DiGraph(graph)
        
    return list(nx.strongly_connected_components(graph))


mine = components(graph)
reference = nx_components(graph)

assert len(mine) == len(reference)

for component in mine:
    assert component in reference